In [10]:
import ROOT
from ROOT import TH1D, TF1
    
def runfit(filename, caption, category, signal_amp):
    ## Open the file

    datadir = "./data/"
    path = datadir + filename

    outputdir = "./output/"

    file = ROOT.TFile.Open(path)

    ## Get histograms
    Continuum_Bkg = file.Get("Continuum_Bkg")
    GGHH = file.Get("GGHH")
    GGH  = file.Get("GGH")
    VBFH = file.Get("VBFH")
    VH   = file.Get("VH")
    ttH  = file.Get("ttH")
    tHq  = file.Get("tHq")

    # define single H bkg histo
    hSH = TH1D("hSH", "Single H Bkg.", 80, 100, 180)
    hSH.Add(GGH)
    hSH.Add(VBFH)
    hSH.Add(VH)
    hSH.Add(ttH)
    hSH.Add(tHq)

    # define full bkg histo
    hFB = TH1D("hFB", "Full Bkg.", 80, 100, 180)
    hFB.Add(hSH, Continuum_Bkg)

    # define signal + full bkg histo
    hSB = TH1D("hSB", "Signal + Bkg.", 80, 100, 180)
    hSB.Add(hFB, GGHH)

    # create histo that contains only the Higgs peak and fit it
    hpeak = TH1D("hpeak", "Single H + signal", 80, 100, 180)
    hpeak.Add(hSH, GGHH)

    peak_fit = ROOT.TF1("peak_fit", "gaus", 100, 180, 3)
    hpeak.Fit(peak_fit, "0 S WL")

    # Fit signal + background model using the parameters of the peak fit
    fit = ROOT.TF1("fit", "exp([0]+[1]*x)+[2]*exp(-0.5*((x-[3])/[4])^2)", 100, 180)
    fit.FixParameter(2, peak_fit.GetParameter(0))
    fit.FixParameter(3, peak_fit.GetParameter(1))
    fit.FixParameter(4, peak_fit.GetParameter(2))
    fit.SetLineColor(8)
    fit.SetLineStyle(1)
    fit.SetLineWidth(3)
    hSB.Fit("fit", "0 S WL", "", 100, 180)

    #signal fit
    signal_peak = ROOT.TF1("signal_peak", "gaus", 100, 180, 3)
    GGHH.Scale(signal_amp)
    GGHH.Fit(signal_peak, "0 S WL")

    signalFit = ROOT.TF1("signal", "[0]*exp(-0.5*((x-[1])/[2])^2)", 100, 180)
    signalFit.FixParameter(0, signal_peak.GetParameter(0))
    signalFit.FixParameter(1, signal_peak.GetParameter(1))
    signalFit.FixParameter(2, signal_peak.GetParameter(2))
    signalFit.SetLineColor(2)
    signalFit.SetLineWidth(3)
    GGHH.Fit("signalFit")

    ## Create canvas with two pads for the main plot and the pseudodata - bkg
    c = ROOT.TCanvas("c", "", 700, 600)
    pad = ROOT.TPad("pad", "", 0, 0, 1, 1)
    for p in [pad]:
        p.SetLeftMargin(0.14)
        p.SetRightMargin(0.05)
        p.SetTickx(False)
        p.SetTicky(False)

    pad.Draw()
    pad.cd()

    # create pseudodata histo and fill with "fit"
    hPD = TH1D("hPD", "Pseudo data", 20, 100, 180)

    for i in range(int(hSB.Integral()/4)):
        hPD.Fill(fit.GetRandom());

    # draw pseudodata
    for h in [hPD]:
        h.SetMarkerStyle(20)
        h.SetMarkerSize(1.2)
        h.SetLineWidth(1)
        h.SetLineColor(ROOT.kBlack)
        h.GetXaxis().SetLabelSize(0.04)
        h.GetXaxis().SetTitleSize(0.04)
        h.GetXaxis().SetTitleOffset(1.2)
        h.GetXaxis().SetTitle("Di-photon Invariant mass [GeV]")
        h.GetYaxis().SetLabelSize(0.04)
        h.GetYaxis().SetTitleSize(0.04)
        #h.GetYaxis().SetTitleOffset(1)
        h.GetYaxis().SetTitle("Events")
        h.SetMinimum(-0.2)
        h.SetStats(0)
        h.SetTitle("")
        h.Draw("E")

    # Draw fit
    fit.Draw("SAME")
    signalFit.Draw("SAME")

    # create cont bkg fit function with the expo parameters of the "fit" and draw
    cont_bkg_fit = ROOT.TF1("cont_bkg_fit", "exp([0]+[1]*x)", 100, 180)
    for i in range(2):
        cont_bkg_fit.SetParameter(i, fit.GetParameter(i))
    cont_bkg_fit.SetLineColor(4)
    cont_bkg_fit.SetLineStyle(2)
    cont_bkg_fit.SetLineWidth(3)
    cont_bkg_fit.Draw("SAME")

    # Add legend
    legend = ROOT.TLegend(0.6, 0.68, 0.85, 0.88)
    legend.SetTextFont(42)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.045)
    legend.AddEntry(cont_bkg_fit, "Continuum Bkg.", "l")
    legend.AddEntry(fit, "Signal + Full Bkg.", "l")
    legend.AddEntry(signalFit, f"Signal x {signal_amp}", "l")
    legend.AddEntry(hPD, "Pseudo-data" ,"lep")
    legend.Draw()

    # Add CMS label
    text = ROOT.TLatex()
    text.SetNDC()

    text.SetTextFont(72)
    text.SetTextSize(0.05)
    text.DrawLatex(0.15, 0.91, "CMS")

    text.SetTextFont(42)
    text.SetTextSize(0.04)
    text.DrawLatex(0.24, 0.91, "Phase-2 Simulation Preliminary")

    text.SetTextSize(0.045)
    text.DrawLatex(0.7, 0.91, "3000 fb^{-1} (14TeV)")

    text.SetTextSize(0.045)
    text.DrawLatex(0.2, 0.85, caption)
    text.DrawLatex(0.2, 0.79, category)

    get_ipython().run_line_magic('jsroot', 'on')
    c.Draw()

    c.SaveAs(outputdir + filename.rstrip(".root")+"_FIT.pdf")

    print("Loop finished")

In [11]:
file_dict = {
    "Inv_mass_gghasOneL_DNN_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(l#nu) #gamma#gamma", "Cat. 1", 1000],
    "Inv_mass_gghasOneL_DNN_2_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(l#nu) #gamma#gamma", "Cat. 2", 100],
    "Inv_mass_gghasOneL_DNN_3_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(l#nu) #gamma#gamma", "Cat. 3", 50],
    "Inv_mass_gghasOneL_DNN_4_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(l#nu) #gamma#gamma", "Cat. 4", 10],
    "Inv_mass_gghasOneL_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(l#nu) #gamma#gamma", "", 1000],
    "Inv_mass_gghasTwoL_HL.root": ["pp#rightarrowHH#rightarrowW(l#nu)W(l#nu) #gamma#gamma", "", 10],
    "Inv_mass_gghasZeroL_HL.root": ["pp#rightarrowHH#rightarrowW(qq)W(qq) #gamma#gamma", "", 1000],
    "Mgg_c3_DNN_HL.root": ["pp#rightarrowHH#rightarrow#tau#tau #gamma#gamma, 1#tau", "Cat. 1", 1000],
    "Mgg_c3_DNN_2_HL.root": ["pp#rightarrowHH#rightarrow#tau#tau #gamma#gamma, 1#tau", "Cat. 2", 100],
    "Mgg_c3_HL.root": ["pp#rightarrowHH#rightarrow#tau#tau #gamma#gamma, 1#tau", "", 1000],
    "Mgg_c4_Zveto_HL.root": ["pp#rightarrowHH#rightarrow#tau#tau #gamma#gamma, 2#tau", "", 100],
}

In [12]:
for file in file_dict:
    runfit(file, file_dict[file][0], file_dict[file][1], file_dict[file][2])

Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished
Loop finished

****************************************
Minimizer is Minuit / Migrad
MinFCN                    =      254.187
Chi2                      =      6979.47
NDf                       =           77
Edm                       =  4.21663e-08
NCalls                    =           94
Constant                  =      78.3233   +/-   3.28845     
Mean                      =      125.129   +/-   0.131013    
Sigma                     =      3.89153   +/-   0.0547096    	 (limited)

****************************************
Minimizer is Minuit / Migrad
MinFCN                    =      5401.91
Chi2                      =      108.929
NDf                       =           78
Edm                       =  1.65962e-08
NCalls                    =           83
p0                        =        11.42   +/-   0.124965    
p1                        =   -0.0

Info in <TCanvas::Print>: pdf file ./output/Inv_mass_gghasOneL_DNN_HL_FIT.pdf has been created
Warning in <ROOT::Math::Fitter::ApplyWeightCorrection>: Covariance matrix for weighted likelihood is not accurate, the errors may be not reliable
Warning in <ROOT::Math::Fitter::ApplyWeightCorrection>: Covariance matrix for weighted likelihood was forced to be defined positive
Info in <TCanvas::Print>: pdf file ./output/Inv_mass_gghasOneL_DNN_2_HL_FIT.pdf has been created
Warning in <ROOT::Math::Fitter::ApplyWeightCorrection>: Covariance matrix for weighted likelihood is not accurate, the errors may be not reliable
Error in <TMinuitMinimizer::GetHessianMatrix>: Hessian matrix has not been computed - status 1
Error in <ROOT::Math::Fitter::ApplyWeightCorrection>: Error retrieving Hesse on weight2 likelihood - cannot compute errors
Warning in <Fit>: Abnormal termination of minimization.
Warning in <ROOT::Math::Fitter::ApplyWeightCorrection>: Covariance matrix for weighted likelihood is not accur